In [190]:
import os

# Import the files module only if in Google Colab
try:
    from google.colab import files
    is_colab = True
except ImportError:
    is_colab = False

In [191]:
def validate_grid_size(grid_size):
    """
    Validate the grid size to be a positive integer.
    """
    if grid_size <= 0:
        raise ValueError("Grid size must be a positive integer.")
    return grid_size

In [192]:
def validate_positions(cells, grid_size):
    """
    Validate that all cell positions are within the bounds of the grid.
    """
    for row, col in cells:
        if not (0 <= row < grid_size and 0 <= col < grid_size):
            raise ValueError(f"Invalid position ({row + 1}, {col + 1}). It is out of bounds.")
    return True

In [193]:
def parse_input_from_file(filename):
    """
    Parses the Calcudoku puzzle input from a file.

    Args:
        filename (str): Path to the input file.

    Returns:
        grid_size (int): Size of the grid.
        cages (list): List of cages, where each cage is a tuple of (clue, operation, cells).
        fixed_cells (dict): Dictionary of fixed cells with their assigned values.
    """
    with open(filename, 'r') as file:
        # Read first line for grid size and verify it's a valid input
        first_line = file.readline().strip().split()
        grid_size = int(first_line[0])

        # Validate grid size
        validate_grid_size(grid_size)
        
        # Read number of cages
        num_cages = int(file.readline().strip())

        cages = []
        fixed_cells = {}

        print(f"Grid Size: {grid_size}")
        print(f"Number of Cages: {num_cages}")

        # Read each cage
        for _ in range(num_cages):
            # Read cage details
            cage_line = file.readline().strip()
            
            # Parse clue and operation
            if cage_line and cage_line[-1] in "+-":
                clue = int(cage_line[:-1])
                operation = cage_line[-1]
                positions_line = file.readline().strip().split()
            else:
                clue = int(cage_line)
                operation = None
                positions_line = file.readline().strip().split()

            cells = []
            for j in range(0, len(positions_line), 2):
                row, col = int(positions_line[j]), int(positions_line[j + 1])
                cells.append((row - 1, col - 1))  # Convert to 0-based indexing
            
            # Validate positions
            validate_positions(cells, grid_size)

            print(f"Cage: Clue: {clue}, Operation: {operation}, Cells in 0-based indexing: {cells}")

            # If a cage has only one cell and it's fixed, treat it as fixed
            if len(cells) == 1:
                fixed_cells[cells[0]] = clue
            
            cages.append((clue, operation, cells))

        return grid_size, cages, fixed_cells

In [194]:
def is_cage_valid(grid, clue, operation, cells):
    """
    Checks if the values in a cage satisfy the cage's clue and operation.
    """
    values = [grid[r][c] for r, c in cells if grid[r][c] != 0]  # Get assigned values

    if operation == '+':  # Addition
        result = sum(values) <= clue and (len(values) < len(cells) or sum(values) == clue)
        print(f"Checking addition cage with clue {clue}: {'Valid' if result else 'Invalid'}")
        return result

    elif operation == '-':  # Subtraction
        if len(values) == len(cells):
            values.sort(reverse=True)  # Sort in descending order
            result = abs(values[0] - values[1]) == clue
            print(f"Checking subtraction cage with clue {clue}: {'Valid' if result else 'Invalid'}")
            return result
        return True

    elif operation is None:  # If there's no operation (just a number like "4")
        # Just return True because there's no operation constraint
        print(f"Checking fixed value in cage with clue {clue}: Always valid")
        return True

    return False

In [195]:
def is_valid_assignment(grid, row, col, value, cages, fixed_cells):
    """
    Checks if placing a value in a specific cell is valid.
    """
    grid_size = len(grid)

    # Skip fixed cells that are already filled with a specific value
    if (row, col) in fixed_cells:
        return value == fixed_cells[(row, col)]

    # Check row uniqueness
    if value in grid[row]:
        return False

    # Check column uniqueness
    if value in [grid[r][col] for r in range(grid_size)]:
        return False

    # Check cage constraints
    for clue, operation, cells in cages:
        if (row, col) in cells:
            # Temporary assignment to test cage validity
            grid[row][col] = value
            if not is_cage_valid(grid, clue, operation, cells):
                grid[row][col] = 0  # Undo the temporary assignment
                return False
            grid[row][col] = 0  # Undo the temporary assignment

    # Log the valid assignment
    print(f"Placed {value} in cell ({row + 1}, {col + 1})")
    return True



In [196]:
def solve(grid, cages, grid_size, fixed_cells):
    """
    Solves the Calcudoku puzzle using backtracking with a fixed exploration order.
    """
    # Get a list of empty cells in row-major order (top-to-bottom, left-to-right)
    empty_cells = [(row, col) for row in range(grid_size) for col in range(grid_size)
                   if grid[row][col] == 0 and (row, col) not in fixed_cells]

    # If no empty cells, puzzle is solved
    if not empty_cells:
        return True

    # Try to fill each empty cell in the consistent order (row-major order)
    row, col = empty_cells[0]
    for value in range(1, grid_size + 1):  # Try values from 1 to grid_size
        if is_valid_assignment(grid, row, col, value, cages, fixed_cells):
            grid[row][col] = value  # Assign value

            if solve(grid, cages, grid_size, fixed_cells):  # Recur
                return True

            grid[row][col] = 0  # Backtrack
            print(f"Backtracked from cell ({row + 1}, {col + 1})")

    return False  # No valid solution found

In [197]:
def solve_calcudoku(input_file):
    """
    Solves a Calcudoku puzzle from an input file and returns the solution as a string.
    """
    # Parse input from file
    grid_size, cages, fixed_cells = parse_input_from_file(input_file)

    # Initialize an empty grid
    grid = [[0] * grid_size for _ in range(grid_size)]

    # Fill in the fixed cells
    for (row, col), value in fixed_cells.items():
        grid[row][col] = value

    # Solve the puzzle
    if solve(grid, cages, grid_size, fixed_cells):
        # Return the solution as a string (for downloading as .txt)
        solution = '\n'.join([' '.join(map(str, row)) for row in grid])
        return solution
    else:
        return "No Solution Exists"

In [198]:
def upload_input_files():
    """
    Upload input files for solving puzzles in Google Colab.
    """
    if is_colab:
        uploaded = files.upload()
        input_files = list(uploaded.keys())  # Get all uploaded files
        return input_files
    else:
        # For local, get predefined files
        input_files = "input1.txt,input2.txt,input3.txt".split(',')
        return [file.strip() for file in input_files]

In [199]:
def download_output_file(solution, output_file):
    """
    Download the solution to a file in .txt format.
    """
    with open(output_file, 'w') as f:
        f.write(solution)

    if is_colab:
        files.download(output_file)  # Trigger file download
    else:
        print(f"Solution: {solution}")
        print(f"Solution written to {output_file}\n")

In [200]:
def main():
    # Get the input files
    input_files = upload_input_files()

    # Process each input file
    for idx, input_file in enumerate(input_files, 1):
        try:
            print(f"Solving for {input_file}")

            # Solve the puzzle for this input file
            solution = solve_calcudoku(input_file)

            # Create corresponding output file name
            output_file = f'output{idx}.txt'

            # Get the solution for this input file
            download_output_file(solution, output_file)

        except ValueError as e:
            print(f"Error processing {input_file}: {e}")
            continue

In [201]:
if __name__ == "__main__":
    main()

Solving for input1.txt
Grid Size: 4
Number of Cages: 9
Cage: Clue: 7, Operation: +, Cells in 0-based indexing: [(0, 0), (1, 0), (2, 0)]
Cage: Clue: 4, Operation: None, Cells in 0-based indexing: [(0, 1)]
Cage: Clue: 1, Operation: -, Cells in 0-based indexing: [(0, 2), (0, 3)]
Cage: Clue: 2, Operation: -, Cells in 0-based indexing: [(1, 1), (1, 2)]
Cage: Clue: 2, Operation: None, Cells in 0-based indexing: [(1, 3)]
Cage: Clue: 3, Operation: None, Cells in 0-based indexing: [(3, 0)]
Cage: Clue: 1, Operation: -, Cells in 0-based indexing: [(2, 1), (3, 1)]
Cage: Clue: 1, Operation: -, Cells in 0-based indexing: [(2, 2), (3, 2)]
Cage: Clue: 3, Operation: -, Cells in 0-based indexing: [(2, 3), (3, 3)]
Checking addition cage with clue 7: Valid
Placed 1 in cell (1, 1)
Placed 2 in cell (1, 3)
Checking subtraction cage with clue 1: Valid
Placed 3 in cell (1, 4)
Checking addition cage with clue 7: Valid
Placed 4 in cell (2, 1)
Placed 1 in cell (2, 2)
Checking subtraction cage with clue 2: Valid
P